In [52]:
import pandas as  pd
import openpyxl
import xlsxwriter
from datetime import datetime

In [53]:
df = pd.read_excel('Austeller.xlsx', names=['Firma','Halle','nummer','fidn','kürzel','fname','pidn','pname'], dtype='string')
df = df.apply(lambda x: x.str.strip())
df.head()

,Firma,Halle,nummer,fidn,kürzel,fname,pidn,pname
0,13 edition Alexandra Frohloff,Halle 2,G534,<NA>,13 edition,13 edition Alexandra Frohloff,<NA>,<NA>
1,1a-faksimile,Halle 2,E403,<NA>,1a-faksimile,1a-faksimile,<NA>,<NA>
2,3. Oktober - Deutschland singt und klingt e.V.,Halle 4,B322,<NA>,3. Oktober - Deutschland singt und klingt e.V.,3. Oktober - Deutschland singt und klingt e.V.,<NA>,<NA>
3,360 GRAD Verlag,Halle 3,B308,<NA>,360 GRAD Verlag,360 GRAD Verlag,<NA>,<NA>
4,3L Verlag,Halle 2,K107,!1064130828!,3L-Verlag,3L-Verlag,<NA>,<NA>


In [57]:
#erstelle mit diesen Daten einen Qd-Datensatz

#erstelle eine Exceltablle mit den benötigten Spalten für die Erschließung der Materialien

def pica_schreiben(row):

    pica =f"""0500 Qd
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy   
1110 $a27.04.2023$b30.04.2023$n27.04.2023 - 30.04.2023$4lfzt
"""

    if pd.notna(row.pidn):
        pica += f"3000 {row.pidn}$BGeistiger Schöpfer$4cre\n"
    else:
        if pd.notna(row.pname):
            pica += f"3000 {row.pname}$BGeistiger Schöpfer$4cre\n"

    if pd.notna(row.fidn):
        pica += f"3100 {row.fidn}$BHerausgebendes Organ$4isb\n"
    else:
        if pd.notna(row.fname):
         pica += f"3100 {row.fname}$BHerausgebendes Organ$4isb\n"

    pica += f"4000 Messestand "

    if pd.notna(row.pname):
        pename = row.pname.split(', ')
        pica +=f"{pename[1]} {pename[0]}"
    else:
        pica +=f"{row.fname}"

    pica += f" auf der Leipziger Buchmesse 2023 / "

    if pd.notna(row.pname):
        pename = row.pname.split(', ')
        pica +=f"{pename[1]} {pename[0]}\n"
    else:
        pica +=f"{row.fname}\n"

    pica +=f"""4046 Leipzig
4700 |BSM|

\t\n"""


    return pica

In [58]:
for index, row in df.iterrows():
    print(pica_schreiben(row))

0500 Qd
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy   
1110 $a27.04.2023$b30.04.2023$n27.04.2023 - 30.04.2023$4lfzt
3100 13 edition Alexandra Frohloff$BHerausgebendes Organ$4isb
4000 Messestand 13 edition Alexandra Frohloff auf der Leipziger Buchmesse 2023 / 13 edition Alexandra Frohloff
4046 Leipzig
4700 |BSM|

	

0500 Qd
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy   
1110 $a27.04.2023$b30.04.2023$n27.04.2023 - 30.04.2023$4lfzt
3100 1a-faksimile$BHerausgebendes Organ$4isb
4000 Messestand 1a-faksimile auf der Leipziger Buchmesse 2023 / 1a-faksimile
4046 Leipzig
4700 |BSM|

	

0500 Qd
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy   
1110 $a27.04.2023$b30.04.2023$n27.04.2023 - 30.04.2023$4lfzt
3100 3. Oktober - Deutschland singt und klingt e.V.$BHerausgebendes Organ$4isb
4000 Messestand 3. Oktober - Deutschland singt und klingt e.V. auf der Leipziger Buchmesse 2023 / 3. Oktober - Deutschland singt 

In [31]:
with open(f"qd_stand-recent.dat", 'w') as f:
    for index, row in df.iterrows():
        f.write(pica_schreiben(row))
        

In [56]:
now = datetime.now()

with open(f"../dat/zk-{now.strftime('%y-%m-%d-%H-%M-%S')}.dat","w") as f:
    with open(f"../dat/zk-recent.dat", 'w') as f2:
        for index, row in df.iterrows():
            f.write(pica_schreiben(row))
            f2.write(pica_schreiben(row))
